# Hull Tactical Market Prediction - PROMETHEUS v2 Submission

Ensemble of LightGBM + XGBoost with Kelly-inspired position sizing + PROMETHEUS features.

**PROMETHEUS Features (+32% Sharpe improvement):**
- Variance Compression (regime shift detection)
- Market Temperature (system energy)
- Order Parameter (coherence measure)
- Critical Slowing Down (AC1 trends)
- Volatility Regime
- Sentiment-Vol Interaction
- Momentum Regime
- Economic Surprise Integration
- Interest Rate Regime

In [ ]:
import os
import pickle
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import polars as pl
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import RobustScaler

warnings.filterwarnings('ignore')

# Paths - PROMETHEUS v2 artifacts
if os.path.exists('/kaggle/input/hull-tactical-market-prediction'):
    DATA_DIR = Path('/kaggle/input/hull-tactical-market-prediction')
    # Try v2 artifacts first, fall back to v1
    ARTIFACTS_DIR = Path('/kaggle/input/hull-artifacts-v2')
    if not ARTIFACTS_DIR.exists():
        ARTIFACTS_DIR = Path('/kaggle/input/hull-artifacts')
    if not ARTIFACTS_DIR.exists():
        ARTIFACTS_DIR = Path('/kaggle/working')
else:
    DATA_DIR = Path('/home/user/aimo3/hull/hull-tactical-market-prediction')
    ARTIFACTS_DIR = Path('/home/user/aimo3/hull/artifacts_v2')  # Use v2!

print(f"Data: {DATA_DIR}")
print(f"Artifacts: {ARTIFACTS_DIR}")

In [ ]:
# Global state
class InferenceState:
    scaler = None
    feature_cols = None
    lgb_models = None
    xgb_models = None
    config = None
    recent_data = None
    history = []
    initialized = False

def initialize():
    """Load all model artifacts."""
    if InferenceState.initialized:
        return
    
    print("Loading PROMETHEUS v2 model artifacts...")
    
    with open(ARTIFACTS_DIR / 'scaler.pkl', 'rb') as f:
        InferenceState.scaler = pickle.load(f)
    
    with open(ARTIFACTS_DIR / 'feature_cols.pkl', 'rb') as f:
        InferenceState.feature_cols = pickle.load(f)
    
    with open(ARTIFACTS_DIR / 'lgb_models.pkl', 'rb') as f:
        InferenceState.lgb_models = pickle.load(f)
    
    InferenceState.xgb_models = []
    i = 0
    while (ARTIFACTS_DIR / f'xgb_model_{i}.json').exists():
        model = xgb.Booster()
        model.load_model(str(ARTIFACTS_DIR / f'xgb_model_{i}.json'))
        InferenceState.xgb_models.append(model)
        i += 1
    
    with open(ARTIFACTS_DIR / 'config.pkl', 'rb') as f:
        InferenceState.config = pickle.load(f)
    
    InferenceState.recent_data = pd.read_parquet(ARTIFACTS_DIR / 'recent_data.parquet')
    
    InferenceState.initialized = True
    print(f"Loaded {len(InferenceState.lgb_models)} LGB + {len(InferenceState.xgb_models)} XGB models")
    print(f"Features: {len(InferenceState.feature_cols)}")
    print(f"Config: {InferenceState.config}")

In [ ]:
def add_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add rolling and PROMETHEUS features."""
    df = df.copy()
    
    # === BASIC ROLLING FEATURES ===
    key_cols = ['V1', 'V2', 'V3', 'M1', 'M2', 'S1', 'S2', 'E1', 'P1', 'I1']
    key_cols = [c for c in key_cols if c in df.columns]
    
    for col in key_cols:
        for window in [5, 21, 63]:
            if len(df) >= window:
                df[f'{col}_ma{window}'] = df[col].rolling(window, min_periods=1).mean()
                df[f'{col}_std{window}'] = df[col].rolling(window, min_periods=1).std().fillna(0)
    
    # Lagged returns if available
    if 'lagged_forward_returns' in df.columns:
        df['lagged_ret'] = df['lagged_forward_returns']
    elif 'forward_returns' in df.columns:
        df['lagged_ret'] = df['forward_returns'].shift(1).fillna(0)
    else:
        df['lagged_ret'] = 0
    
    # Return-based features
    if 'lagged_ret' in df.columns:
        for window in [5, 10, 21]:
            if len(df) >= window:
                df[f'ret_cumsum_{window}'] = df['lagged_ret'].rolling(window, min_periods=1).sum()
                df[f'ret_vol_{window}'] = df['lagged_ret'].rolling(window, min_periods=1).std().fillna(0)
    
    # === PROMETHEUS INSIGHT #1: Variance Compression ===
    if 'lagged_ret' in df.columns and len(df) >= 63:
        df['var_21'] = df['lagged_ret'].rolling(21, min_periods=1).var().fillna(0)
        df['var_63'] = df['lagged_ret'].rolling(63, min_periods=1).var().fillna(0)
        df['var_ratio'] = df['var_21'] / (df['var_63'] + 1e-8)
        df['var_compression'] = (df['var_ratio'] < 0.5).astype(int)
    else:
        df['var_21'] = 0
        df['var_63'] = 0
        df['var_ratio'] = 1.0
        df['var_compression'] = 0
    
    # === PROMETHEUS INSIGHT #2: Market Temperature ===
    v_cols = [c for c in df.columns if c.startswith('V') and c[1:].isdigit()]
    if v_cols:
        df['v_mean'] = df[v_cols].mean(axis=1)
        df['v_std'] = df[v_cols].std(axis=1)
        df['temperature'] = df['v_std'] / (df['v_mean'].abs() + 1e-8)
    else:
        df['v_mean'] = 0
        df['v_std'] = 0
        df['temperature'] = 0
    
    # === PROMETHEUS INSIGHT #3: Order Parameter (simplified) ===
    # Measure coherence within feature groups
    for prefix in ['V', 'M', 'S']:
        cols = [c for c in df.columns if c.startswith(prefix) and c[1:].isdigit()]
        if len(cols) >= 2:
            # Simple order parameter: variance of z-scores
            zscores = (df[cols] - df[cols].mean()) / (df[cols].std() + 1e-8)
            df[f'{prefix}_order'] = 1 - zscores.var(axis=1).fillna(1)
        else:
            df[f'{prefix}_order'] = 0
    
    # === PROMETHEUS INSIGHT #11: Critical Slowing Down (AC1) ===
    if 'lagged_ret' in df.columns and len(df) >= 63:
        def calc_ac1(x):
            if len(x) < 10:
                return 0
            try:
                return np.corrcoef(x[:-1], x[1:])[0, 1]
            except:
                return 0
        df['ac1'] = df['lagged_ret'].rolling(63, min_periods=10).apply(calc_ac1, raw=True).fillna(0)
        df['ac1_rising'] = (df['ac1'] > df['ac1'].shift(5).fillna(0)).astype(int)
    else:
        df['ac1'] = 0
        df['ac1_rising'] = 0
    
    # === PROMETHEUS INSIGHT #13: Volatility Regime ===
    if v_cols:
        df['vol_regime'] = df[v_cols].mean(axis=1)
        df['vol_regime_ma21'] = df['vol_regime'].rolling(21, min_periods=1).mean()
        df['vol_expanding'] = (df['vol_regime'] > df['vol_regime_ma21']).astype(int)
    else:
        df['vol_regime'] = 0
        df['vol_regime_ma21'] = 0
        df['vol_expanding'] = 0
    
    # === PROMETHEUS INSIGHT #14: Sentiment-Volatility Interaction ===
    s_cols = [c for c in df.columns if c.startswith('S') and c[1:].isdigit()]
    if s_cols:
        df['sent_mean'] = df[s_cols].mean(axis=1)
        df['sent_vol_interact'] = df['sent_mean'] / (df['vol_regime'] + 1e-8)
    else:
        df['sent_mean'] = 0
        df['sent_vol_interact'] = 0
    
    # === PROMETHEUS INSIGHT #16: Momentum Regime ===
    if 'ret_cumsum_21' in df.columns:
        df['cum_ret_std'] = df['ret_cumsum_21'].rolling(63, min_periods=1).std().fillna(0)
        df['momentum_strong'] = (df['ret_cumsum_21'].abs() > df['cum_ret_std']).astype(int)
    else:
        df['cum_ret_std'] = 0
        df['momentum_strong'] = 0
    
    # === PROMETHEUS INSIGHT #17: Economic Surprise ===
    e_cols = [c for c in df.columns if c.startswith('E') and c[1:].isdigit()]
    if e_cols:
        df['econ_mean'] = df[e_cols].mean(axis=1)
        df['econ_momentum'] = df['econ_mean'].diff(5).fillna(0)
        df['econ_surprise'] = df['econ_momentum'] - df['econ_momentum'].rolling(63, min_periods=1).mean()
    else:
        df['econ_mean'] = 0
        df['econ_momentum'] = 0
        df['econ_surprise'] = 0
    
    # === PROMETHEUS INSIGHT #18: Interest Rate Regime ===
    i_cols = [c for c in df.columns if c.startswith('I') and c[1:].isdigit()]
    if len(i_cols) >= 3 and 'I3' in df.columns and 'I1' in df.columns:
        df['rate_slope'] = df['I3'] - df['I1']
        df['rate_slope_pct'] = df['rate_slope'].rolling(63, min_periods=1).rank(pct=True).fillna(0.5)
        df['rate_inverting'] = (df['rate_slope_pct'] < 0.1).astype(int)
    else:
        df['rate_slope'] = 0
        df['rate_slope_pct'] = 0.5
        df['rate_inverting'] = 0
    
    return df


def predict(test: pl.DataFrame) -> float:
    """Main prediction function for evaluation API."""
    # Initialize on first call
    if not InferenceState.initialized:
        initialize()
    
    # Convert to pandas
    test_pd = test.to_pandas()
    
    # Add to history
    InferenceState.history.append(test_pd)
    if len(InferenceState.history) > 300:
        InferenceState.history = InferenceState.history[-300:]
    
    # Combine with historical data for feature calculation
    if len(InferenceState.history) < 63:
        # Use recent training data for initial predictions
        n_needed = 300 - len(InferenceState.history)
        combined = pd.concat(
            [InferenceState.recent_data.tail(n_needed)] + InferenceState.history,
            ignore_index=True
        )
    else:
        combined = pd.concat(InferenceState.history, ignore_index=True)
    
    # Add PROMETHEUS features
    combined = add_features(combined)
    
    # Ensure all feature columns exist
    for col in InferenceState.feature_cols:
        if col not in combined.columns:
            combined[col] = 0
    
    # Get current row features
    X = combined[InferenceState.feature_cols].iloc[[-1]].fillna(0)
    
    # Scale
    X_scaled = pd.DataFrame(
        InferenceState.scaler.transform(X),
        columns=InferenceState.feature_cols
    )
    
    # Predict with ensemble
    predictions = []
    
    for model in InferenceState.lgb_models:
        pred = model.predict(X_scaled)
        predictions.append(pred[0])
    
    dtest = xgb.DMatrix(X_scaled)
    for model in InferenceState.xgb_models:
        pred = model.predict(dtest)
        predictions.append(pred[0])
    
    predictions = np.array(predictions)
    mean_pred = predictions.mean()
    std_pred = predictions.std()
    
    # Position sizing with PROMETHEUS config
    cfg = InferenceState.config
    uncertainty = max(std_pred, 1e-5)
    kelly = mean_pred / (cfg['risk_aversion'] * uncertainty**2 + 1e-8)
    position = cfg['base_position'] + cfg['scale_factor'] * kelly
    position = np.clip(position, cfg['min_position'], cfg['max_position'])
    
    return float(position)

In [ ]:
# Import evaluation API
import kaggle_evaluation.default_inference_server

# Create inference server
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)

print("PROMETHEUS v2 inference server ready.")

In [ ]:
# Run inference
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    print("Running local test with PROMETHEUS v2 features...")
    inference_server.run_local_gateway((str(DATA_DIR),))
    print("\n✅ Local test completed!")